# PDF 링크 추출

카카오 & 3,4,5 페이지에 해당하는 PDF 링크 모두 추출하기

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin

# 전체 열 너비 제한 해제
pd.set_option("display.max_colwidth", None)

# 기본 URL과 타겟 페이지 목록
base_url = "https://finance.naver.com"
pages = [3, 4, 5]

# 결과 저장용 리스트
pdf_links = []

# 페이지별 크롤링
for page in pages:
    url = f"{base_url}/research/company_list.naver?searchType=itemCode&itemName=카카오&itemCode=035720&page={page}"
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "html.parser")

    for a_tag in soup.find_all("a", href=True):
        href = a_tag['href']
        if href.endswith(".pdf"):
            full_link = urljoin(base_url, href)
            pdf_links.append(full_link)

# DataFrame으로 정리
df = pd.DataFrame(pdf_links, columns=["PDF 링크"])
print(df)


                                                                                PDF 링크
0   https://stock.pstatic.net/stock-research/company/34/20240809_company_235285000.pdf
1   https://stock.pstatic.net/stock-research/company/57/20240809_company_570019000.pdf
2   https://stock.pstatic.net/stock-research/company/18/20240809_company_569300000.pdf
3   https://stock.pstatic.net/stock-research/company/63/20240809_company_109899000.pdf
4    https://stock.pstatic.net/stock-research/company/2/20240809_company_576437000.pdf
..                                                                                 ...
82          https://ssl.pstatic.net/imgstock/upload/research/company/1683519812535.pdf
83          https://ssl.pstatic.net/imgstock/upload/research/company/1683518317748.pdf
84          https://ssl.pstatic.net/imgstock/upload/research/company/1683517856310.pdf
85          https://ssl.pstatic.net/imgstock/upload/research/company/1683514752823.pdf
86          https://ssl.pstatic.net/imgstoc

# PDF plumber 사용하여 데이터 전처리

### 라이브러리 설치 및 경고메시지 무시

In [3]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 78.0 MB/s eta 0:00:00


In [4]:
import os
import pdfplumber as pp
import pandas as pd

In [11]:
import logging
logging.getLogger("pdfminer").setLevel(logging.ERROR)

### pdf 파일 불러오기

In [9]:
import os
import requests

# ➊ 저장 폴더 생성
folder_path = "/content/pdf 저장소"
os.makedirs(folder_path, exist_ok=True)

# ➋ 저장된 개수 카운트용 변수
saved_count = 0

# ➌ 첫 번째 컬럼 기준으로 링크 반복
for i, url in enumerate(df.iloc[:, 0], 1):  # 첫 번째 열 기준
    file_name = f"kakao_report_{i}.pdf"
    save_path = os.path.join(folder_path, file_name)

    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(response.content)
            saved_count += 1
    except:
        pass  # 오류는 무시하고 넘어감

# ✅ 마지막에 한 번만 출력
print(f"총 {saved_count}개 저장 완료 ✅")

총 87개 저장 완료 ✅


In [13]:
import pdfplumber

# PDF 경로 설정
file_path = "/content/pdf 저장소/kakao_report_1.pdf"

# PDF 열기
with pdfplumber.open(file_path) as pdf:
    # 전체 페이지 순회하며 텍스트 추출
    for i, page in enumerate(pdf.pages):
        text = page.extract_text()
        print(f"\n--- Page {i+1} ---\n")
        print(text)




--- Page 1 ---

Company Report
카카오
035720
Aug 9, 2024
Buy 톡비즈 성장 재가속 필요
유지
TP 51,000원 하향
2Q24 Review: 매출 & 영익 컨센서스 부합
2Q24 매출은 2조 49억원(YoY +4%), 영익은 1,340억원으로 시장 예상치(매출 2조 511억
Company Data
원, 영익 1,332억원)에 부합. 톡비즈 광고 매출은 톡채널 활성 광고주 수 및 전체 친구 수
현재가(08/08) 38,450 원
액면가(원) 100 원 확대로 메시지 비즈니스 성장(YoY +16%), 친구탭 비즈보드와 오픈채팅탭 채팅목록형 광
52주 최고가(보통주) 61,100 원
고 매출 증가에 따른 DA(YoY +5%) 성장으로 3,073억원(YoY +9%) 기록. 거래형 매출은
52주 최저가(보통주) 36,300 원
KOSPI (08/08) 2,556.73p 전 분기와 유사한 정도의 커머스 통합거래액 성장 이어지며 2,066억원(YoY +5%). 플랫폼
KOSDAQ (08/08) 745.28p
자본금 445 억원 기타 모빌리티 전 부문 견조 성장과 페이 결제 및 금융서비스 성장으로 3,535억원(YoY
시가총액 170,504 억원
+18%) 기록. 게임 매출은 기존 게임 매출 하향으로 2,335억원(YoY -13%), 뮤직 매출은
발행주식수(보통주) 44,349 만주
발행주식수(우선주) 0 만주 주요 소속 아티스트 신보 발매와 활발한 콘서트 활동으로 5,109억원(YoY +6%) 기록.
평균거래량(60일) 151.9 만주
평균거래대금(60일) 604 억원
영업비용 중 인건비는 보수적인 인력 채용 기조 유지로 YoY +2%, 매출연동비는 매출 성
외국인지분(보통주) 26.74%
주요주주 장 추이에 따라 YoY +2%, 매출 대비 매출연동비 비중은 39.7%. 외주인프라비는 브레인
김범수 외 92 인 24.22%
MAXIMO PTE 6.31% 영업양수도에 따른 인프라수수료 감소로 YoY -5%, 상각비는 자체, 신규임차

In [14]:
# 1페이지만 보기
file_path = "/content/pdf 저장소/kakao_report_1.pdf"  # 네가 저장한 PDF 경로

with pdfplumber.open(file_path) as pdf:
    first_page = pdf.pages[0]             # 1페이지는 인덱스 0
    text = first_page.extract_text()      # 텍스트 추출
    print(text)

Company Report
카카오
035720
Aug 9, 2024
Buy 톡비즈 성장 재가속 필요
유지
TP 51,000원 하향
2Q24 Review: 매출 & 영익 컨센서스 부합
2Q24 매출은 2조 49억원(YoY +4%), 영익은 1,340억원으로 시장 예상치(매출 2조 511억
Company Data
원, 영익 1,332억원)에 부합. 톡비즈 광고 매출은 톡채널 활성 광고주 수 및 전체 친구 수
현재가(08/08) 38,450 원
액면가(원) 100 원 확대로 메시지 비즈니스 성장(YoY +16%), 친구탭 비즈보드와 오픈채팅탭 채팅목록형 광
52주 최고가(보통주) 61,100 원
고 매출 증가에 따른 DA(YoY +5%) 성장으로 3,073억원(YoY +9%) 기록. 거래형 매출은
52주 최저가(보통주) 36,300 원
KOSPI (08/08) 2,556.73p 전 분기와 유사한 정도의 커머스 통합거래액 성장 이어지며 2,066억원(YoY +5%). 플랫폼
KOSDAQ (08/08) 745.28p
자본금 445 억원 기타 모빌리티 전 부문 견조 성장과 페이 결제 및 금융서비스 성장으로 3,535억원(YoY
시가총액 170,504 억원
+18%) 기록. 게임 매출은 기존 게임 매출 하향으로 2,335억원(YoY -13%), 뮤직 매출은
발행주식수(보통주) 44,349 만주
발행주식수(우선주) 0 만주 주요 소속 아티스트 신보 발매와 활발한 콘서트 활동으로 5,109억원(YoY +6%) 기록.
평균거래량(60일) 151.9 만주
평균거래대금(60일) 604 억원
영업비용 중 인건비는 보수적인 인력 채용 기조 유지로 YoY +2%, 매출연동비는 매출 성
외국인지분(보통주) 26.74%
주요주주 장 추이에 따라 YoY +2%, 매출 대비 매출연동비 비중은 39.7%. 외주인프라비는 브레인
김범수 외 92 인 24.22%
MAXIMO PTE 6.31% 영업양수도에 따른 인프라수수료 감소로 YoY -5%, 상각비는 자체, 신규임차 데이터센터
운영 및 AI 사업

In [15]:
# 1페이지만 보기
file_path = "/content/pdf 저장소/kakao_report_2.pdf"  # 네가 저장한 PDF 경로

with pdfplumber.open(file_path) as pdf:
    first_page = pdf.pages[0]             # 1페이지는 인덱스 0
    text = first_page.extract_text()      # 텍스트 추출
    print(text)

2024년 8월 9일 I 기업분석_Earnings Review
카카오
BUY (유지) (035720)
하반기 변화 예고
목표주가(12M) 60,000원
현재주가(8.08) 38,450원
Key Data 2Q24 Re: 비즈니스 메시지가 이끈 본업
KOSPI 지수 (pt) 2,556.73
52주 최고/최저(원) 61,100/36,300
카카오는 2분기 연결 기준 영업수익 2조 49억원(+4.2%YoY, +0.8%QoQ), 영업이익 1,340
시가총액(십억원) 17,050.4
시가총액비중(%) 0.81 억원(+18.5%YoY, +11.4%QoQ, OPM 6.7%)으로 컨센서스에 부합하는 실적을 기록했다. 비
발행주식수(천주) 443,491.8
60일 평균 거래량(천주) 1,292.7 우호적인 경기 상황에도 톡비즈 광고의 견조한 성장이 나타났다. 2분기 플랫폼 부문 매출액
60일 평균 거래대금(십억원) 54.1
외국인지분율(%) 26.79 은 9,548억원(+12.6%YoY, -0.2%QoQ)이다. 톡비즈 매출액은 5,139억원(+7.0%YoY, -
주요주주 지분율(%)
1.6%QoQ)를 기록했다. 광고 부문에서 비즈보드(+5%YoY)의 성장은 소폭 둔화되었으나 타
김범수 외 92 인 24.22
MAXIMO PTE 6.31
겟팅에 특화된 비즈니스 메시지(+16%YoY)는 여전히 높은 성장률을 보이고 있다. 톡채널
Consensus Data 활성 광고주 수와 전체 친구 수가 +11% 증가했고, 이미지/동영상이 발송 가능한 프리미엄
2024 2025
매출액(십억원) 8,410.5 9,216.3 상품 채택률이 높아져 P가 증가한 영향이다. 영업비용은 1조 8,709억원(+3.4%YoY,
영업이익(십억원) 586.6 730.8
+0.2%QoQ)을 기록했다. 일본 내 경쟁 심화로 픽코마의 마케팅비가 증가했으나 브레인 영
순이익(십억원) 386.3 543.1
EPS(원) 849 1,092 업양수도에 따른 인프라수수료의 일시적인 감소로 안정적인 수준을 유지했다.
BPS(원) 2

다음 스탭

In [16]:
# 데이터 프레임 만들 것임.

# 2024 <- 1페이지에서 가장 위에 이거 포함되어있는 텍스트 추출해서 "date"라는 칼럼에 저장
#  "buy" "hold" "sell" 텍스트 있는 (대문자도 포함) 그 줄 추출해서 "투자의견"
# TP 혹은 목표주가 목표 주가 라는 텍스트 있으면 그 줄 추출해서 "목표주가"
# "증권" 포함된 단어 출력해서 "증권사"

# 이렇게 저장해서 하나의 행을 만들고 이걸 저기있는 모든 PDF 대상으로 실행해줘
